<a href="https://colab.research.google.com/github/barthelemymp/TULIP-TCR/blob/main/tulip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Clone github repo
import json, time, os, sys, glob

if not os.path.isdir("TULIP-TCR"):
  os.system("git clone -q https://github.com/barthelemymp/TULIP-TCR.git")
sys.path.append('/content/TULIP-TCR/')

In [2]:
!pip install transformers==4.24.0  datasets  tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
import pandas as pd
import sys

import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data._utils.collate import default_collate
#from data import TranslationDataset
from transformers import BertTokenizerFast, BertTokenizer
from transformers import BertModel, BertForMaskedLM, BertConfig, EncoderDecoderModel, BertLMHeadModel, AutoModelForSequenceClassification
from sklearn.metrics import roc_auc_score

import sys
import torch
import torch.utils.data as data
from torch.nn.utils.rnn import pad_sequence
import os


from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertOnlyMLMHead, SequenceClassifierOutput
from torch.nn import MSELoss, CrossEntropyLoss, BCEWithLogitsLoss
from typing import List, Optional, Tuple, Union
from transformers.modeling_outputs import ModelOutput

from transformers import PretrainedConfig
from transformers.modeling_outputs import BaseModelOutput, Seq2SeqLMOutput
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import add_start_docstrings, add_start_docstrings_to_model_forward, logging, replace_return_docstrings
from transformers.models.encoder_decoder.configuration_encoder_decoder import EncoderDecoderConfig
import warnings

from src.multiTrans import ED_BertForSequenceClassification, TCRDataset, BertLastPooler, unsupervised_auc, train_unsupervised, eval_unsupervised, MyMasking, Tulip, get_logscore


# Load model and file

In [4]:
from google.colab import files
uploaded = files.upload()
filesname = list(uploaded.keys())[0]
finetunedmodel = True #@param {type:"boolean"}
with open("/content/TULIP-TCR/configs/shallow.config.json", "r") as read_file:
    print("loading hyperparameter")
    modelconfig = json.load(read_file)


Saving VDJ_test_2.csv to VDJ_test_2.csv
loading hyperparameter


# run prediction

In [5]:
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)



tokenizer = AutoTokenizer.from_pretrained("/content/TULIP-TCR/aatok/")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<PAD>'})

if tokenizer.sep_token is None:
    tokenizer.add_special_tokens({'sep_token': '<MIS>'})

if tokenizer.cls_token is None:
    tokenizer.add_special_tokens({'cls_token': '<CLS>'})

if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({'eos_token': '<EOS>'})

if tokenizer.mask_token is None:
    tokenizer.add_special_tokens({'mask_token': '<MASK>'})



from tokenizers.processors import TemplateProcessing
tokenizer._tokenizer.post_processor = TemplateProcessing(
    single="<CLS> $A <EOS>",
    pair="<CLS> $A <MIS> $B:1 <EOS>:1",
    special_tokens=[
        ("<EOS>", 2),
        ("<CLS>", 3),
        ("<MIS>", 4),
    ],
)

mhctok = AutoTokenizer.from_pretrained("/content/TULIP-TCR/mhctok/")

vocabsize = len(tokenizer._tokenizer.get_vocab())
mhcvocabsize = len(mhctok._tokenizer.get_vocab())
print("Loading models ..")

max_length = 114
encoder_config = BertConfig(vocab_size = vocabsize,
                    max_position_embeddings = max_length,
                    num_attention_heads = modelconfig["num_attn_heads"],
                    num_hidden_layers = modelconfig["num_hidden_layers"],
                    hidden_size = modelconfig["hidden_size"],
                    type_vocab_size = 1,
                    pad_token_id =  tokenizer.pad_token_id)

encoder_config.mhc_vocab_size  =mhcvocabsize

encoderA = BertModel(config=encoder_config)
encoderB = BertModel(config=encoder_config)
encoderE = BertModel(config=encoder_config)

max_length = 50
decoder_config = BertConfig(vocab_size = vocabsize,
                    max_position_embeddings = max_length,
                    num_attention_heads = modelconfig["num_attn_heads"],
                    num_hidden_layers = modelconfig["num_hidden_layers"],
                    hidden_size = modelconfig["hidden_size"],
                    type_vocab_size = 1,
                    is_decoder=True,
                    pad_token_id =  tokenizer.pad_token_id)    # Very Important

decoder_config.add_cross_attention=True

decoderA = ED_BertForSequenceClassification(config=decoder_config)
decoderA.pooler = BertLastPooler(config=decoder_config)
decoderB = ED_BertForSequenceClassification(config=decoder_config)
decoderB.pooler = BertLastPooler(config=decoder_config)
decoderE = ED_BertForSequenceClassification(config=decoder_config)
decoderE.pooler = BertLastPooler(config=decoder_config)
# Define encoder decoder model
model = Tulip(encoderA=encoderA,encoderB=encoderB,encoderE=encoderE, decoderA=decoderA, decoderB=decoderB, decoderE=decoderE)

def count_parameters(mdl):
    return sum(p.numel() for p in mdl.parameters() if p.requires_grad)


for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_normal_(p)

if finetunedmodel:
    checkpoint = torch.load("/content/TULIP-TCR/model_weights/hla0201_finetuned/multiTCR_s_mhcXfintune2/pytorch_model.bin")
    model.load_state_dict(checkpoint)
    print("loaded")
else:
    checkpoint = torch.load("/content/TULIP-TCR/model_weights/pretrained/multiTCR_s_mhcX_2_below20out/pytorch_model.bin")
    model.load_state_dict(checkpoint)
    print("loaded")


model.to(device)




Using pad_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using device: cuda
Loading models ..
self.pad_token_id 1
self.pad_token_id 1
self.pad_token_id 1
loaded


Tulip(
  (encoderA): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30, 128, padding_idx=1)
      (position_embeddings): Embedding(114, 128)
      (token_type_embeddings): Embedding(1, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)

False

In [6]:
filesname = "/content/TULIP-TCR/data/VDJ_test_2.csv"
compute_auc=True
from sklearn.metrics import roc_auc_score

In [11]:
target_peptidesFinal = pd.read_csv(filesname)["peptide"].unique()

for target_peptide in target_peptidesFinal:
    results = pd.DataFrame(columns=["CDR3a", "CDR3b", "peptide", "rank"])
    datasetPetideSpecific= TCRDataset(filesname, tokenizer, device,target_peptide=target_peptide, mhctok=mhctok)
    print(target_peptide)
    scores = -1*np.array(get_logscore(datasetPetideSpecific, model, ignore_index =  tokenizer.pad_token_id))
    ranks = np.argsort(np.argsort(scores))
    results["CDR3a"] = datasetPetideSpecific.alpha
    results["CDR3b"] = datasetPetideSpecific.beta
    results["peptide"] = target_peptide
    results["rank"] = ranks
    # print(results)
    if compute_auc:
      dl = torch.utils.data.DataLoader(dataset=datasetPetideSpecific, batch_size=1, shuffle=False, collate_fn=datasetPetideSpecific.all2allmhc_collate_function)
      # print(unsupervised_auc(model,dl, tokenizer.pad_token_id))
      auce = roc_auc_score(datasetPetideSpecific.binder, ranks)
      print(auce)




Loading the data ...
GILGFVFTL
0.9469207100591716
Loading the data ...
LLWNGPMAV
0.8813688740802443
Loading the data ...
CINGVCWTV
0.8489892984542212
Loading the data ...
ELAGIGILTV
0.8803677721088436
Loading the data ...
WLLWPVTLA
0.375
Loading the data ...
NLVPMVATV
0.9138052983539094
Loading the data ...
YLQPRTFLL
0.8972336534064929
Loading the data ...
SLFNTVATLY
0.8299319727891156
Loading the data ...
RMFPNAPYL
0.75
Loading the data ...
TLLFLMSFT
0.20833333333333331
Loading the data ...
LLYDANYFL
0.8843537414965986
Loading the data ...
GLCTLVAML
0.922699933169971
Loading the data ...
ALWEIQQVV
0.5374149659863945
Loading the data ...
RLCAYCCNI
1.0
Loading the data ...
FVAAIFYLI
0.5
Loading the data ...
FLASKIGRLV
0.7395833333333334
Loading the data ...
LLFNKVTLA
0.8333333333333334
Loading the data ...
HLMSFPQSA
0.5416666666666666
Loading the data ...
YLNTLTLAV
1.0
Loading the data ...
ILGFVFTLT
0.45833333333333337
Loading the data ...
KLQFTSLEI
0.8333333333333334
Loading the data .

KeyboardInterrupt: ignored